In [1]:
import sympy
sp = sympy
import numpy as np
from scipy.integrate import odeint,RK45,solve_ivp
import matplotlib.pyplot as plt

# Functions to Derive Affine Form

In [2]:
def get_g(a,b,c):
    return sp.sqrt(b**2 - 4.0*a*c)
    
def get_Riccati_solution(a,b,c,tau,f0):
    g = get_g(a,b,c)
    ex = sp.exp(g*tau) - 1
    num = 2*g*f0 + ((b+g)*f0+2*c)*ex
    denom = 2*g - (2*a*f0+b-g)*ex
    return num/denom

def get_Riccati_solution_vec(a,b,c,tau,f0):
    n = len(a)
    res = np.zeros(n)
    for i in range(n):
        res[i] = get_Riccati_solution(a[i],b[i],c[i],tau,f0[i])
    return res

def get_int_Riccati_solution(a,b,c,tau,riccati_f0):
    g = get_g(a,b,c)
    ex = sp.exp(g*tau) - 1
    term1 = (-b+g)*tau/(2.0*a) 
    term2 = sp.log(2*g/sp.Abs(2*g+(g-b-2*a*riccati_f0)*ex))/a
    return term1 + term2        

def get_int_Riccati_solution_vec(a,b,c,tau,riccati_f0):
    n = len(a)
    res = np.zeros(n)
    for i in range(n):
        res[i] = get_int_Riccati_solution(a[i],b[i],c[i],tau,riccati_f0[i])
    return res

def get_E_vAvB(t,alp,beta,sigma,v0):
    res = np.exp(-beta*t)*(v0 - alp/beta) + alp/beta
    return res.prod()

In [3]:
a, b, c, tau, f0 = sp.symbols("a b c tau f0")

In [4]:
g = sp.symbols('g')
riccati_res = get_Riccati_solution(a,b,c,tau,f0)
#riccati_res = riccati_res.subs(get_g(a,b,c),g)
#riccati_res = riccati_res.subs(sp.sqrt(-a*c+0.25*b*b),g/4)
riccati_res = riccati_res.simplify()
riccati_res

(4.0*f0*sqrt(-a*c + 0.25*b**2) - (1 - exp(2.0*tau*sqrt(-a*c + 0.25*b**2)))*(2*c + f0*(b + 2.0*sqrt(-a*c + 0.25*b**2))))/((1 - exp(2.0*tau*sqrt(-a*c + 0.25*b**2)))*(2*a*f0 + b - 2.0*sqrt(-a*c + 0.25*b**2)) + 4.0*sqrt(-a*c + 0.25*b**2))

In [5]:
# Print the expression
inp = {a:0.005,b:-0.25,c:2,f0:1.0,tau:30}
riccati_res.subs(inp)

9.92289389208465

# Affine Form

In [6]:
# correlation parameters
sAsB = 0.15 # It is important 
sAvA = -0.5
sAvB = -0.5
sBvA = -0.5
sBvB = -0.5
vAvB = 0.5
# sA, sB, vA, vB check the sequense always
correlation = np.matrix([[1,sAsB,sAvA,sAvB],
                         [sAsB,1,sBvA,sBvB],
                         [sAvA,sBvA,1,vAvB],
                         [sAvB,sBvB,vAvB,1]])
# correlation = np.matrix([[1,sAvA,sAsB,sAvB],
#                          [sAvA,1,vAsB,vAvB],
#                          [sAsB,vAsB,1,sBvB],
#                          [sAvB,vAvB,sBvB,1]])
if np.sum(np.linalg.eigvals(correlation) < 0):
    print('Correlation matrix is not positive definite!')
L = np.linalg.cholesky(correlation) # Cholesky decomposition of the correlation for simulation

params = {
# chf params    
    'tau': 30,
    'w': np.array([1,1,1]), # weights for chf
    'u': np.array([1.0,1.0]), # weights for price
    'psi': np.array([1.0,1.0]), # weights for vol
# treasury params
    'l0': -0.127061, # constant shift
    've': np.array([0.2715618,0.0195524,0.0009720]),
    'ka': np.array([5.6772530,0.2520333,0.147]),
    'si': np.array([0.0181427,0.0422960,0.034]), # adjusted si[2]
    'x0': np.array([0.05095958,0.06725220,0.00961570]),
# Equity parameters (US LC, OS)
    's0': np.array([0.0,0.0]), # initial value
    'mu0': np.array([0.000815,0.000822]), # official value
    'mu0': np.array([0.07,0.110]), # my value
    'mu1': np.array([-0.5,-0.5]), # I think just -0.5 is too strict? High volatility -> Always stock crush
    # mu1 = np.array([-0.001,-0.002]) + np.array([-0.5,-0.5]) # I think just -0.5 is too strict?
    'v0': np.array([0.019528084,0.080837794]),
    'alpha': np.array([0.022,0.108045]),
    'beta': np.array([0.98803,1.284485]),
    'sigma': np.array([0.142303,0.121922]), # volatility of variance
    'correlation': correlation,
    'L': L
}
params["th"] = params["ve"]/params["ka"]

In [13]:
# compute chf
p = params
u = p["u"]
tau = p["tau"]
correlation = p["correlation"]
sAsB = p["correlation"][0,1]
sBvA = p["correlation"][1,2]
sAvB = p["correlation"][0,3]
sigma = p["sigma"]
# compute parameters
B_a = 0.5*p['si']**2
B_b = -p['ka']
B_c = p['u'].sum() * np.ones(len(p['x0']))
B_f0 = p['w']
C_a = 0.5*sigma**2
C_b = p['u']*np.array([p['correlation'][0,3],p['correlation'][1,3]])*p['sigma']-p['beta']
C_c = 0.5*p['u']**2 + p['u']*p['mu1']
C_f0 = p['psi']
E_vAvB = get_E_vAvB(30.0,p["alpha"],p["beta"],p["sigma"],p["v0"]) # presume

# compute A
A = 0.0
A += ((u*p["mu0"]).sum() + p["l0"]*u.sum()+E_vAvB*u.prod()*sAsB)*tau
A += (p["ka"]*p["th"]*get_int_Riccati_solution_vec(B_a,B_b,B_c,tau,B_f0)).sum()
A += ((p["alpha"] + np.array([u[1]*sBvA,u[0]*sAvB])*sigma*E_vAvB)*get_int_Riccati_solution_vec(C_a,C_b,C_c,tau,C_f0)).sum()
# compute B
B =  get_Riccati_solution_vec(B_a,B_b,B_c,tau,B_f0)
# compute C
C = get_Riccati_solution_vec(C_a,C_b,C_c,tau,C_f0)
# Compute ChF
np.exp(A + np.dot(u,p["s0"]) + np.dot(B,p["x0"]) + np.dot(C,p["v0"]))

414.4711822554628

In [12]:
print(A,B,C)

5.323564771519748 [ 0.35228665  8.16686377 14.16919632] [1.60055688e-14 2.97038380e-18]


# Appendicies

In [ ]:
int_riccati_res = get_int_Riccati_solution(a,b,c,tau,f0)
int_riccati_res

In [ ]:
inp = {a:0.005,b:-0.25,c:2,f0:1.0,tau:10}
int_riccati_res.subs(inp)

In [ ]:
int_riccati_res_sp = sp.integrate(riccati_res,tau)
int0 = int_riccati_res_sp.subs(tau,0).expand()
int_riccati_res_sp -= int0
# int_riccati_res_sp.subs(inp) - int0.subs(inp)

In [ ]:
dfda = sp.diff(riccati_res,a)
dfda = dfda.subs(get_g(a,b,c),g)
dfda = dfda.subs(sp.sqrt(-a*c+0.25*b*b),g/4)
dfda = dfda.simplify()
dfda

In [ ]:
g = sp.symbols('g')
riccati_res.subs(sp.sqrt(b**2 - 4.0*a*c),g)

In [ ]:
str(riccati_res)

In [ ]:
print(sp.python(riccati_res))